In [1]:
%load_ext autoreload
%autoreload 2

In [362]:
import numpy as np
from src.onnx_model import YoloOnnxModel
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as T
from torch import nn
import cv2
import torch
from ultralytics import YOLO
import onnxruntime as ort
import torchvision
from src.utils.vision import non_maximum_supression
from src.visualization import plot_yolo_labels
from src.transforms import xywh2xyxy, xywhn2xywh

onnx_model = YoloOnnxModel("../models/detection_model.onnx")

/home/shate/.cache/pypoetry/virtualenvs/digits-detection-hppNHGvS-py3.11/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


---
# **yolo outputs to nms ONNX**

In [358]:
# outputs_transposed = outputs.transpose(0, 2, 1)          # V

# boxes_xywh = outputs_transposed[..., :4]      # V
# class_scores = outputs_transposed[..., 4:]    # V

# scores = class_scores.max(axis=2, keepdims=True) # V
# scores = scores.transpose(0, 2, 1)  # V

# class_ids = class_scores.argmax(axis=2).squeeze()

# iou_threshold = np.array([0.7], dtype=np.float32)
# score_threshold = np.array([0.25], dtype=np.float32)

# pad_x, pad_y = pad
# w = onnx_model.input_w - pad_x * 2
# h = onnx_model.input_h - pad_y * 2

# boxes_xywh[:, :, 0] -= pad_x
# boxes_xywh[:, :, 1] -= pad_y

# boxes_xywhn = np.divide(boxes_xywh, np.array([w, h, w, h]))

In [367]:
image = Image.open("../datasets/yolo_HWD+/images/test/42.png")
# image = Image.open("../datasets/SVHN/examples/5.png")

image_ = np.asarray(image)
image_ = image_[100:-50, 40:-50]
img = np.copy(image_)

input_tensor, pad = onnx_model.prepare_input(img)
outputs = onnx_model.inference(input_tensor)[0]

In [368]:
from src.onnx_nms import create_onnx_NMS

create_onnx_NMS(filepath="nms.onnx", opset=18)
nms = ort.InferenceSession("nms.onnx", providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

CONF_THRESHOLD = 0.25
IOU_THRESHOLD = 0.7
MAX_OUTPUT_BOXES_PER_CLASS = 100

nms_results = nms.run(
    ["selected_boxes_xywh", "selected_class_scores", "selected_class_ids"], 
    {
        "output0": outputs, 
        "max_output_boxes_per_class": np.array([MAX_OUTPUT_BOXES_PER_CLASS], dtype=np.int32),
        'iou_threshold': np.array([IOU_THRESHOLD], dtype=np.float32), 
        'score_threshold': np.array([CONF_THRESHOLD], dtype=np.float32)
    }
)
nms_results

[array([[      186.4,      62.237,      29.219,      59.791],
        [      62.26,      186.25,      37.847,      60.953],
        [     215.28,      111.26,      26.218,       59.98]], dtype=float32),
 array([    0.99841,     0.99451,     0.86349], dtype=float32),
 array([9, 5, 0], dtype=int32)]

---
# **preprocessing onnx**

In [369]:
input_h, input_w, input_c = 256, 256, 3



In [359]:
# img = img[..., :input_c]    # V

# img_h, img_w, img_c = img.shape   # V
# aspect_ratio = img_w / img_h  # V
# if aspect_ratio > 1:  # V
#     new_img_h = int(input_w / aspect_ratio)  # V
#     new_img_w = input_w  # V
# else:  # V
#     new_img_h = input_h  # V
#     new_img_w = int(input_h / aspect_ratio)  # V
    
# resized_img = cv2.resize(img, (new_img_w, new_img_h), interpolation=cv2.INTER_LINEAR)  # V

# # width, height ratios
# padded_img = np.ones((input_h, input_w, img_c)) * fill_value
# left = (input_w - new_img_w) // 2
# bottom = (input_h - new_img_h) // 2

# padded_img[bottom : bottom + new_img_h, left : left + new_img_w] = resized_img
# pad_x = (input_w - new_img_w) // 2
# pad_y = (input_h - new_img_h) // 2

# pad = (pad_x, pad_y)
# img = padded_img

# img = img / 255.0
# img = img.transpose(2, 0, 1)
# input_tensor = img.astype(np.float32)
# # input_tensor = np.expand_dims(img, 0).astype(np.float32)

In [444]:
from src.onnx_preprocessing import create_onnx_preprocessing

create_onnx_preprocessing("preprocessing.onnx")
preprocessing = ort.InferenceSession("preprocessing.onnx", providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

In [445]:
INPUT_H = 256
INPUT_W = 256
FILL_VALUE = 114

inputs = {
    "image": image_, 
    "input_h": np.array([INPUT_H], dtype=np.int32), 
    "input_w": np.array([INPUT_W], dtype=np.int32),
    "fill_value": np.array([FILL_VALUE], dtype=np.uint8)
}
preprocessed = preprocessing.run(None, inputs)

2023-06-12 21:50:37.590253853 [W:onnxruntime:, execution_frame.cc:857 VerifyOutputSizes] Expected shape from model of {3,-1,3} does not match actual shape of {3,256,256} for output preprocessed_img


In [446]:
preprocessed[0].shape

(3, 256, 256)

In [447]:
preprocessed[1]

array([14,  0, 14,  0], dtype=int32)

In [ ]:
%timeit onnx_model(img, 0.7, 0.25)